In [3]:
!pip install pandas==2.2.2 numpy pyspark==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 53.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=c0b56cdea97d861145b0fc29cca85a85b3fdc4bf582b0e7e3c21fbed02636c08
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is 

In [4]:
# 1
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('Eka1', 'Pengusaha', 3000000),
        ('Ekaa2', 'IT Consultant', 1000000),
        ('Ekaaa3', 'Programmer', 2000000),
        ('Ekaaaa4', 'Data Analyst', 1500000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+-------------+-------+
|EmployeeName|   Department| Salary|
+------------+-------------+-------+
|        Eka1|    Pengusaha|3000000|
|       Ekaa2|IT Consultant|1000000|
|      Ekaaa3|   Programmer|2000000|
|     Ekaaaa4| Data Analyst|1500000|
+------------+-------------+-------+



In [5]:
# 2 Contoh operasi transformasi DataFrame
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 3000).show()
df.groupBy('Department').avg('Salary').show()

+------------+-------+
|EmployeeName| Salary|
+------------+-------+
|        Eka1|3000000|
|       Ekaa2|1000000|
|      Ekaaa3|2000000|
|     Ekaaaa4|1500000|
+------------+-------+

+------------+-------------+-------+
|EmployeeName|   Department| Salary|
+------------+-------------+-------+
|        Eka1|    Pengusaha|3000000|
|       Ekaa2|IT Consultant|1000000|
|      Ekaaa3|   Programmer|2000000|
|     Ekaaaa4| Data Analyst|1500000|
+------------+-------------+-------+

+-------------+-----------+
|   Department|avg(Salary)|
+-------------+-----------+
|IT Consultant|  1000000.0|
|    Pengusaha|  3000000.0|
|   Programmer|  2000000.0|
| Data Analyst|  1500000.0|
+-------------+-----------+



In [6]:
# 3 Menambahkan kolom bonus
df_with_bonus = df.withColumn('SalaryBonus', df['Salary'] * 0.1)

# Menambahkan kolom TotalCompensation menggunakan df_with_bonus
df_with_total = df_with_bonus.withColumn('TotalCompensation', df_with_bonus['Salary'] + df_with_bonus['SalaryBonus'])

# Menampilkan DataFrame dengan kolom baru
df_with_total.show()

+------------+-------------+-------+-----------+-----------------+
|EmployeeName|   Department| Salary|SalaryBonus|TotalCompensation|
+------------+-------------+-------+-----------+-----------------+
|        Eka1|    Pengusaha|3000000|   300000.0|        3300000.0|
|       Ekaa2|IT Consultant|1000000|   100000.0|        1100000.0|
|      Ekaaa3|   Programmer|2000000|   200000.0|        2200000.0|
|     Ekaaaa4| Data Analyst|1500000|   150000.0|        1650000.0|
+------------+-------------+-------+-----------+-----------------+



In [7]:
# 4 Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+-------------+-------+----+
|EmployeeName|   Department| Salary|Rank|
+------------+-------------+-------+----+
|     Ekaaaa4| Data Analyst|1500000|   1|
|       Ekaa2|IT Consultant|1000000|   1|
|        Eka1|    Pengusaha|3000000|   1|
|      Ekaaa3|   Programmer|2000000|   1|
+------------+-------------+-------+----+

